In [47]:
import os
import sys
import time
import pandas as pd
import numpy as np
import win32com.client
from pywinauto import application
from PyQt5.QtWidgets import *


In [48]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [49]:
import os
import win32com.client
from pywinauto import application
import time
import pandas as pd

# from datetime import time


def connect(reconnect=True):
    # 재연결이라면 기존 연결을 강제로 kill
    if reconnect:
        try:
            os.system('taskkill /IM ncStarter* /F /T')
            os.system('taskkill /IM CpStart* /F /T')
            os.system('taskkill /IM DibServer* /F /T')
            os.system('wmic process where "name like \'%ncStarter%\'" call terminate')
            os.system('wmic process where "name like \'%CpStart%\'" call terminate')
            os.system('wmic process where "name like \'%DibServer%\'" call terminate')
        except:
            pass

    CpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

    if CpCybos.IsConnect:
        print('already connected.')

    else:
        app = application.Application()
        app.start(
            'C:\Daishin\Starter\\ncStarter.exe /prj:cp /id:{id} /pwd:{pwd} /pwdcert:{pwdcert} /autostart'.format(
                id='ssuyane', pwd='13579', pwdcert='qkrtndus1!!')
        )
        # 연결 될때까지 무한루프
        while True:
            if CpCybos.IsConnect:
                break
            time.sleep(1)

        print('connected.')
    return CpCybos


def _get_ohlcv_by_date(StockChart, code, last_date, start_date, chart_gb, modified='1', field=(0, # 날짜
                                                                                               2, # 시가
                                                                                               3, # 고가
                                                                                               4, # 저가
                                                                                               5, # 종가
                                                                                               8, # 거래량
                                                                                               9, # 거래대금
                                                                                               12, # 상장주식수
                                                                                               13, # 시가총액
                                                                                               16, # 외국인보유수량
                                                                                               17, # 외국인보유비율
                                                                                               20, # 기관순매수
                                                                                               21, # 기관누적순매수
                                                                                               23 # 등락률
                                                                                               )):
    StockChart.SetInputValue(0, code)
    StockChart.SetInputValue(1, ord('1'))
    StockChart.SetInputValue(2, last_date)
    StockChart.SetInputValue(3, start_date)
    StockChart.SetInputValue(5, field)
    StockChart.SetInputValue(6, ord(chart_gb))
    StockChart.SetInputValue(9, ord(modified))
    StockChart.BlockRequest()
    return _get_df_from_stock_chart(StockChart)

def initIndexInfo(self):
    # 지표 기본 정보, 지표 이름 = 지표 라인 리스트
    self.indexList = {}
    self.indexList['지표선택 없음'] =['없음']
    self.indexList['이동평균(라인1개)'] =['이동평균']
    self.indexList['Stochastic Slow'] = ['SLOW K', 'SLOW D']
    self.indexList['MACD'] = ['MACD', 'SIGNAL', 'OSCILLATOR']
    self.indexList['RSI'] = ['RSI', 'SIGNAL']
    self.indexList['Binary Wave MACD'] = ['BWMACD','SIGNAL', 'OSCILLATOR']
    self.indexList['TSF'] = ['TSF','SIGNAL']
    self.indexList['ZigZag'] = ['ZigZag1', 'ZigZag2']
    self.indexList['Bollinger Band'] = ['Bol상', 'Bol하', 'Bol중']


def _get_df_from_stock_chart(StockChart):
    time.sleep(0.25)
    df = []
    num_col = StockChart.GetHeaderValue(1)
    num_row = StockChart.GetHeaderValue(3)
    code = StockChart.GetHeaderValue(0) # 초
    timess = StockChart.GetHeaderValue(18) # 초
    exFlag = StockChart.GetHeaderValue(19) # 예상체결 플래그
    cprice = StockChart.GetHeaderValue(13) # 현재가
    diff = StockChart.GetHeaderValue(2) # 대비
    cVol = StockChart.GetHeaderValue(17) # 순간체결수량
    vol = StockChart.GetHeaderValue(9) # 거래량
    
    for r_idx in range(num_row):
        curr_data = []

        for c_idx in range(num_col):
            curr_data.append(StockChart.GetDataValue(c_idx, r_idx))
        df.append(curr_data)

    df = pd.DataFrame(df, columns=['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'Accumulated sales volume' , 'Accumulated number of contract'])
    return df

def get_ohlcv_by_date(code, last_date, start_date, chart_gb, modified='1', field=(0,1,2,3,4,5,8,10,11)):
   

    print('get ohlcv by date')
    print(f'code: {code}\nstart_date: {start_date}\nlast_date: {last_date}')
    StockChart = win32com.client.Dispatch("CpSysDib.StockChart")

    df = _get_ohlcv_by_date(StockChart, code, last_date, start_date,
                            chart_gb, modified=modified, field=field)
    last_date = df['date'].iloc[-1]
    last_min = df['time'].iloc[-1]

    df_list = [df]

    while True:
        df = _get_ohlcv_by_date(StockChart, code, int(last_date), start_date,
                            chart_gb, modified=modified, field=field)

        curr_date = df['date'].iloc[-1]
        curr_min = df['time'].iloc[-1]

        if curr_date == last_date and curr_min == last_min:
            break

        last_date = curr_date
        last_min = curr_min

        df_list.append(df)

    df = pd.concat(df_list).drop_duplicates(['date', 'time']).reset_index(drop=True)
    
    return df


# 이미 연결되어있다면 재연결 x
CpCybos = connect(False)



kospi200_code = ['A033780', 'A064350', 'A017670', 'A005490', 'A014680', 'A012450', 'A051900', 'A009150', 'A066570', 'A108320', 'A047810', 'A000660', 'A010140', 'A035420', 'A005380', 'A009830', 'A052690', 'A034020', 'A005070', 'A005420', 'A042700', 'A028050', 'A068270', 'A137310', 'A005930', 'A017960', 'A075580', 'A003490']
tablename = ['jusik_past_A033780', 'jusik_past_A064350', 'jusik_past_A017670', 'jusik_past_A005490', 'jusik_past_A014680', 'jusik_past_A012450', 'jusik_past_A051900', 'jusik_past_A009150', 'jusik_past_A066570', 'jusik_past_A108320', 'jusik_past_A047810', 'jusik_past_A000660', 'jusik_past_A010140', 'jusik_past_A035420', 'jusik_past_A005380', 'jusik_past_A009830', 'jusik_past_A052690', 'jusik_past_A034020', 'jusik_past_A005070', 'jusik_past_A005420', 'jusik_past_A042700', 'jusik_past_A028050', 'jusik_past_A068270', 'jusik_past_A137310', 'jusik_past_A005930', 'jusik_past_A017960', 'jusik_past_A075580', 'jusik_past_A003490']

for i, n in zip(kospi200_code, tablename):
    code = i
    name = n
    start_date = '20210620'
    last_date = '20230713'
    chart_gb = 'm'
    df = get_ohlcv_by_date(code, last_date, start_date, chart_gb)
    PastDate = df['date'].to_list()
    CheTime = df['time'].to_list()
    OpenPrice = df['open'].to_list()
    HighPrice = df['high'].to_list()
    LowPrice = df['low'].to_list()
    ClosePrice = df['close'].to_list()
    Volume = df['volume'].to_list()
    ASaleVol = df['Accumulated sales volume'].to_list()
    ANumVol = df['Accumulated number of contract'].to_list()

    for pastdate, chetime, openprice, highprice, lowprice, closeprice, volume, asalevol, anumvol in zip(PastDate, CheTime, OpenPrice, HighPrice, LowPrice, ClosePrice, Volume, ASaleVol, ANumVol):
        sql = f"INSERT INTO {n} (PastDate, Chetime, OpenPrice, HighPrice, LowPrice, ClosePrice, Volume, ASaleVol, ANumVol) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)" % ("'"+str(pastdate)+"'", "'"+str(chetime)+"'", "'"+str(openprice)+"'", "'"+str(highprice)+"'","'"+str(lowprice)+"'","'"+str(closeprice)+"'","'"+str(volume)+"'","'"+str(asalevol)+"'","'"+str(anumvol)+"'")
        cur.execute(sql)
    print(df)
    conn.commit()

already connected.
get ohlcv by date
code: A033780
start_date: 20230712
last_date: 20230712
         date  time   open   high    low  close  volume  \
0    20230712  1530  82000  82000  82000  82000   27074   
1    20230712  1520  81900  82000  81800  82000    1618   
2    20230712  1519  81900  82000  81900  81900    1217   
3    20230712  1518  82000  82000  81900  81900     714   
4    20230712  1517  82000  82000  81900  81900    1010   
..        ...   ...    ...    ...    ...    ...     ...   
375  20230712   905  82000  82000  82000  82000      12   
376  20230712   904  82000  82000  82000  82000      45   
377  20230712   903  82000  82000  81900  82000      38   
378  20230712   902  82000  82000  81800  81900      62   
379  20230712   901  81800  82000  81800  82000     920   

     Accumulated sales volume  Accumulated number of contract  
0                       98750                           78704  
1                       98750                           51664  
2      

In [50]:
# for date,title,url,content in zip(JusikTimess,JusikCprice,JusikDiff,JusikCVol,JusikVol):
#     sql = "INSERT INTO jusik_tic_chart (JusikTimess,JusikCprice,JusikDiff,JusikCVol,JusikVol) VALUES (%s, %s, %s, %s)" % ("'"+date+"'", "'"+title+"'", "'"+url+"'", "'"+content+"'")
#     print(sql)
#     cur.execute(sql)
# conn.commit()
# conn.close()

In [51]:
# PastDate = df['date'].to_list()
# CheTime = df['time'].to_list()
# OpenPrice = df['open'].to_list()
# HighPrice = df['high'].to_list()
# LowPrice = df['low'].to_list()
# ClosePrice = df['close'].to_list()
# Volume = df['volume'].to_list()
# ASaleVol = df['Accumulated sales volume'].to_list()
# ANumVol = df['Accumulated number of contract'].to_list()

# for pastdate, chetime, openprice, highprice, lowprice, closeprice, volume, asalevol, anumvol in zip(PastDate, CheTime, OpenPrice, HighPrice, LowPrice, ClosePrice, Volume, ASaleVol, ANumVol):
#     sql = "INSERT INTO {i} (PastDate, Chetime, OpenPrice, HighPrice, LowPrice, ClosePrice, Volume, ASaleVol, ANumVol) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)" % ("'"+str(pastdate)+"'", "'"+str(chetime)+"'", "'"+str(openprice)+"'", "'"+str(highprice)+"'","'"+str(lowprice)+"'","'"+str(closeprice)+"'","'"+str(volume)+"'","'"+str(asalevol)+"'","'"+str(anumvol)+"'")
#     print(sql)
#     cur.execute(sql)
# conn.commit()